# Tarefa 02
A segunda tarefa consiste em calcular o total líquido da empresa.<br>
Esse total é calculado da seguinte forma total_liquido = soma(total_bruto – desconto_percentual).<br>
O cálculo é baseado no conjunto de dados a seguir:

``` json
transacoes = [{'transacao_id':1, 'total_bruto':3000, 'desconto_percentual':6.99},
              {'transacao_id':2, 'total_bruto':57989, 'desconto_percentual':1.45},
              {'transacao_id':4, 'total_bruto':1, 'desconto_percentual':None},
              {'transacao_id':5, 'total_bruto':34, 'desconto_percentual':0.0}]
```

O resultado esperado é um código com pyspark que retorne o total líquido da empresa que é 59973.46.6.

## Resolução

Para essa tarefa iremos utilizar o Pyspark SQL.

#### Criando sessão do spark:

In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder\
                    .appName('resolucao_tarefa_02')\
                    .getOrCreate()

23/10/29 12:28:26 WARN Utils: Your hostname, NBKPE08TCSB resolves to a loopback address: 127.0.1.1; using 172.29.120.13 instead (on interface eth0)
23/10/29 12:28:26 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/10/29 12:28:28 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


#### Definindo json com os dados

In [2]:
transacoes = [{'transacao_id':1, 'total_bruto':3000, 'desconto_percentual':6.99},
              {'transacao_id':2, 'total_bruto':57989, 'desconto_percentual':1.45},
              {'transacao_id':4, 'total_bruto':1, 'desconto_percentual':None},
              {'transacao_id':5, 'total_bruto':34, 'desconto_percentual':0.0}]

#### Criando Dataframe a partir do json

In [3]:
df = spark.createDataFrame(transacoes)

#### Criando um view temporária
Optei por utilizar o spark SQL, então abaixo teremos uma view temporária para tratar os valores `None` da coluna `desconto_percentual`

In [4]:
df.createOrReplaceTempView('transacoes')

#### Consulta para tratar os valores `None`

In [5]:
dados = spark.sql('''
   select 
       coalesce(desconto_percentual, 0) as desconto_percentual
       , total_bruto
       , transacao_id
   from transacoes    
''')
dados.createOrReplaceTempView('total_liquido')

#### Consulta aplicando o calculo para obter o valor líquido total

In [13]:
total_liquido = spark.sql('''
    select
        round(sum(total_bruto * ((100 - desconto_percentual)/100)), 2) as total_liquido
    from total_liquido
''')
total_liquido.show()

+-------------+
|total_liquido|
+-------------+
|     59973.46|
+-------------+

